# Load stuff

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

from sklearn.cluster import HDBSCAN

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def corpus_from_file(filename):
    corpus = pd.read_csv(filename, sep='\t')
    corpus.drop('feats', axis=1, inplace=True)
    corpus.drop('xpos', axis=1, inplace=True)
    return corpus

dev = corpus_from_file('short_dev.csv')
dev

,sent_id,form,lemma,upos,head_dist,deprel
0,0,و,w,CCONJ,1,cc
1,0,يبلغ,balag-u_1,VERB,0,root
2,0,300,DEFAULT,NUM,2,nummod
3,0,الف,>alof_1,NUM,-1,compound
4,0,دولار,duwlAr_1,NOUN,-3,obj
...,...,...,...,...,...,...
1295,259,ينفذ,naf~a*_1,VERB,0,root
1296,259,ها,hA,PRON,-1,obj
1297,259,الشباب,$ab~_1,NOUN,-2,nsubj
1298,259,في,fiy_1,ADP,1,case


# Compare two sentences

In [3]:
def corpus_get_setence(corpus, sent_id):
    return corpus.loc[corpus['sent_id'] == sent_id].reset_index()

def sentence_compare(corpus, sent_id1, sent_id2):
    sent1 = corpus_get_setence(corpus, sent_id1)
    sent2 = corpus_get_setence(corpus, sent_id2)
    dp = {}

    def _compare(i1, i2):
        if (i1, i2) in dp:
            return dp[(i1, i2)]
        dist = float('inf')
        if i1 < len(sent1.index) or i2 < len(sent2.index):
            if i1 < len(sent1.index) and i2 < len(sent2.index):
                t1 = sent1.loc[i1]
                t2 = sent2.loc[i2]
                if t1['upos'] == t2['upos']:
                    dist = min(dist, _compare(i1+1, i2+1))
            if i1 < len(sent1.index):
                dist = min(dist, 1 + _compare(i1+1, i2))
            if i2 < len(sent2.index):
                dist = min(dist, 1 + _compare(i1, i2+1))
        else:
            dist = 1
        dp[(i1, i2)] = dist
        return dist
    return _compare(0, 0)    

print(corpus_get_setence(dev, 0).to_markdown())
print()
print(corpus_get_setence(dev, 1).to_markdown())
print()
%timeit sentence_compare(dev, 0, 1)

|    |   index |   sent_id | form   | lemma     | upos   |   head_dist | deprel   |
|---:|--------:|----------:|:-------|:----------|:-------|------------:|:---------|
|  0 |       0 |         0 | و      | w         | CCONJ  |           1 | cc       |
|  1 |       1 |         0 | يبلغ   | balag-u_1 | VERB   |           0 | root     |
|  2 |       2 |         0 | 300    | DEFAULT   | NUM    |           2 | nummod   |
|  3 |       3 |         0 | الف    | >alof_1   | NUM    |          -1 | compound |
|  4 |       4 |         0 | دولار  | duwlAr_1  | NOUN   |          -3 | obj      |

|    |   index |   sent_id | form   | lemma     | upos   |   head_dist | deprel   |
|---:|--------:|----------:|:-------|:----------|:-------|------------:|:---------|
|  0 |       5 |         1 | و      | w         | CCONJ  |           2 | iobj     |
|  1 |       6 |         1 | هو     | huwa_1    | PRON   |           1 | nsubj    |
|  2 |       7 |         1 | يصعد   | SaEid-a_1 | VERB   |           0 | ro

In [4]:
dev['sent_id'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

# Test HDBSCAN

In [5]:
# testing my understanding of HDBSCAN distance matrix
hdb = HDBSCAN(metric='precomputed', min_cluster_size=2).fit([
    [0, 1, 1, 5, 5],
    [1, 0, 1, 5, 5],
    [1, 1, 0, 5, 5],
    [5, 5, 5, 0, 1],
    [5, 5, 5, 1, 0],
])
print(hdb.labels_)
print(hdb.probabilities_)

[0 0 0 1 1]
[1. 1. 1. 1. 1.]


# Try HDBSCAN on a small set of sentences

In [6]:
def corpus_first_n_sentences(corpus, N):
    return corpus.loc[corpus['sent_id'].isin(corpus['sent_id'].unique()[:N])]

corpus_first_n_sentences(dev, 3)

,sent_id,form,lemma,upos,head_dist,deprel
0,0,و,w,CCONJ,1,cc
1,0,يبلغ,balag-u_1,VERB,0,root
2,0,300,DEFAULT,NUM,2,nummod
3,0,الف,>alof_1,NUM,-1,compound
4,0,دولار,duwlAr_1,NOUN,-3,obj
5,1,و,w,CCONJ,2,iobj
6,1,هو,huwa_1,PRON,1,nsubj
7,1,يصعد,SaEid-a_1,VERB,0,root
8,1,الى,<ilaY_1,ADP,1,case
9,1,الباص,bAS_1,NOUN,-2,obj


In [7]:
from tqdm.notebook import tqdm

def corpus_distance_matrix(corpus):
    sent_ids = corpus['sent_id'].unique()
    N = len(sent_ids)
    dist = np.zeros((N, N))
    for i in tqdm(range(N)):
        for j in tqdm(range(i, N), leave=False):
            dist[i][j] = dist[j][i] = sentence_compare(corpus, sent_ids[i], sent_ids[j])
    return sent_ids, dist

corpus_distance_matrix(corpus_first_n_sentences(dev, 10))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([[1., 5., 7., 5., 5., 7., 7., 7., 7., 7.],
        [5., 1., 7., 3., 3., 5., 5., 7., 7., 5.],
        [7., 7., 1., 7., 5., 5., 5., 7., 7., 5.],
        [5., 3., 7., 1., 3., 5., 5., 5., 5., 5.],
        [5., 3., 5., 3., 1., 3., 5., 5., 7., 3.],
        [7., 5., 5., 5., 3., 1., 5., 5., 7., 3.],
        [7., 5., 5., 5., 5., 5., 1., 7., 7., 5.],
        [7., 7., 7., 5., 5., 5., 7., 1., 5., 5.],
        [7., 7., 7., 5., 7., 7., 7., 5., 1., 7.],
        [7., 5., 5., 5., 3., 3., 5., 5., 7., 1.]]))

In [17]:
def cluster_corpus(corpus, matrix_fn, min_cluster_size=5):
    print('== Computing distance matrix...')
    %time
    sent_ids, matrix = matrix_fn(corpus)
    print('== Clustering...')
    %time
    hdb = HDBSCAN(metric='precomputed', min_cluster_size=min_cluster_size).fit(matrix)
    print(hdb.labels_)
    print(hdb.probabilities_)

    print('== Aggregating...')
    from collections import defaultdict
    clusters = defaultdict(lambda: [])
    cluster_prob = defaultdict(lambda: 1)
    for prob, sent_id, label in zip(hdb.probabilities_, sent_ids, hdb.labels_):
        clusters[label].append(sent_id)
        cluster_prob[label] *= prob
    print()
    return sorted([(cluster_prob[k], k, v) for k, v in clusters.items()], reverse=True)

def clusters_write_to_file(clusters, corpus, filename):
    print('== Writing to file...')
    with open(filename, 'w') as f:
        for p, label, v in clusters:
            if label >= 0:
                f.write(f"\nCluster {label:02d}:\n")
            else:
                f.write(f"\n\n\nNOT CLUSTERED:\n")
            f.write(f"\t%{p*100:.2f}\n")
            for sent_id in v:
                f.write(' '.join(corpus_get_setence(corpus, sent_id)['form']))
                f.write('\n')

dev_small = corpus_first_n_sentences(dev, 100)
clusters = cluster_corpus(dev_small, corpus_distance_matrix, 2)
clusters_write_to_file(clusters, dev_small, 'shorts_cluster.txt')

== Computing distance matrix...
CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.53 μs


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

== Clustering...
CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.48 μs
[ 6 11 10 11 11 11 12  3 21 11 -1 21 -1 13  8 13  1  1 13  5 13 -1  8 12
 13 13  5  4  4  2 11  2  2  2 10  2  2  2  3  3 -1 -1 11  7  3  9 -1  9
 14 14 16 18 -1 22 23 23 23 16 18 23 -1 17 17 17 -1 -1 -1 -1  6  5  7 19
 20 19 20 20  8 20 20 -1  5  0 13 20 20 11 20 20  0 -1 15 15 15 15 15  5
  7 22 -1 13]
[1.         0.33333333 1.         0.33333333 0.33333333 1.
 1.         0.33333333 1.         0.33333333 0.         1.
 0.         0.33333333 1.         0.33333333 1.         1.
 0.33333333 1.         0.33333333 0.         1.         1.
 1.         0.33333333 0.33333333 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.33333333 1.         0.         0.
 1.         1.         1.         1.         0.         1.
 1.         1.         1.         1.         0.         1.
 1.         1.         1.         1.         1.         1.
 0.         0.33333333 

# Build the matrix in Cython

In [10]:
%load_ext Cython

/usr/lib/python3/dist-packages/pythran/tables.py:4520: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, method):
/usr/lib/python3/dist-packages/pythran/tables.py:4553: FutureWarning: In the future `np.bytes` will be defined as the corresponding NumPy scalar.
  obj = getattr(themodule, elem)


In [11]:
%%cython
def c_corpus_distance_matrix(corpus):
    cdef double dist [10][10]

    def c_corpus_get_sentence(corpus, sent_id):
        return corpus.loc[corpus['sent_id'] == sent_id].reset_index()

    def c_sentence_compare(sent1, sent2):
        cdef double dp [10][10]
        def _compare(i1, i2):
            if dp[i1][i2] != 0:
                return dp[i1][i2]
            dist = float('inf')
            if i1 < len(sent1.index) or i2 < len(sent2.index):
                if i1 < len(sent1.index) and i2 < len(sent2.index):
                    t1 = sent1.iloc[i1]
                    t2 = sent2.iloc[i2]
                    if t1['upos'] == t2['upos']:
                        dist = min(dist, _compare(i1+1, i2+1))
                if i1 < len(sent1.index):
                    dist = min(dist, 1 + _compare(i1+1, i2))
                if i2 < len(sent2.index):
                    dist = min(dist, 1 + _compare(i1, i2+1))
            else:
                dist = 1
            dp[i1][i2] = dist
            return dist
        return _compare(0, 0)

    N = len(corpus)
    for i in range(N):
        for j in range(i, N):
            dist[i][j] = dist[j][i] = c_sentence_compare(corpus[i], corpus[j])
    return dist

In [12]:
def flat_corpus_first_n_sentences(corpus, N):
    ans = []
    ids = corpus['sent_id'].unique()[:N]
    for x in ids:
        ans.append(corpus.loc[corpus['sent_id'] == x])
    return ans

c_corpus_distance_matrix(flat_corpus_first_n_sentences(dev, 10))

[[1.0, 5.0, 7.0, 5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0],
 [5.0, 1.0, 7.0, 3.0, 3.0, 5.0, 5.0, 7.0, 7.0, 5.0],
 [7.0, 7.0, 1.0, 7.0, 5.0, 5.0, 5.0, 7.0, 7.0, 5.0],
 [5.0, 3.0, 7.0, 1.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0],
 [5.0, 3.0, 5.0, 3.0, 1.0, 3.0, 5.0, 5.0, 7.0, 3.0],
 [7.0, 5.0, 5.0, 5.0, 3.0, 1.0, 5.0, 5.0, 7.0, 3.0],
 [7.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 7.0, 7.0, 5.0],
 [7.0, 7.0, 7.0, 5.0, 5.0, 5.0, 7.0, 1.0, 5.0, 5.0],
 [7.0, 7.0, 7.0, 5.0, 7.0, 7.0, 7.0, 5.0, 1.0, 7.0],
 [7.0, 5.0, 5.0, 5.0, 3.0, 3.0, 5.0, 5.0, 7.0, 1.0]]

In [13]:
corpus_distance_matrix(corpus_first_n_sentences(dev, 10))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([[1., 5., 7., 5., 5., 7., 7., 7., 7., 7.],
        [5., 1., 7., 3., 3., 5., 5., 7., 7., 5.],
        [7., 7., 1., 7., 5., 5., 5., 7., 7., 5.],
        [5., 3., 7., 1., 3., 5., 5., 5., 5., 5.],
        [5., 3., 5., 3., 1., 3., 5., 5., 7., 3.],
        [7., 5., 5., 5., 3., 1., 5., 5., 7., 3.],
        [7., 5., 5., 5., 5., 5., 1., 7., 7., 5.],
        [7., 7., 7., 5., 5., 5., 7., 1., 5., 5.],
        [7., 7., 7., 5., 7., 7., 7., 5., 1., 7.],
        [7., 5., 5., 5., 3., 3., 5., 5., 7., 1.]]))

In [14]:
%timeit c_corpus_distance_matrix(flat_corpus_first_n_sentences(dev, 10))

110 ms ± 8.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit corpus_distance_matrix(corpus_first_n_sentences(dev, 10))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

322 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
